In [701]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from urllib.request import urlopen
from PIL import Image
import random

### DATASET :

- Here I have used the MINST image dataset which has already been used in class to segregate the images. Here we are using this dataset to segregate the images from numbers 0 to 9

In [702]:
import numpy as np
#for the following statement to compile successfully, you need the scikit-learn package. 

from sklearn.datasets import load_digits #for loading digits dataset
from sklearn import tree #decision tree from scikit learn 

np.random.seed(1000) #for reproducibility
digits = load_digits()
#check the shape of digits data features
print('digits data shape:', digits.data.shape)
A = digits.data
#check the shape of digits target labels
print('Digits Target:', digits.target[0:180])
y=digits.target
#We can print first 5 samples of digits data and check 
print('\nFeatures of first five samples of digits data:')
print(A[0:5,])

digits data shape: (1797, 64)
Digits Target: [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 9 5 5 6 5 0
 9 8 9 8 4 1 7 7 3 5 1 0 0 2 2 7 8 2 0 1 2 6 3 3 7 3 3 4 6 6 6 4 9 1 5 0 9
 5 2 8 2 0 0 1 7 6 3 2 1 7 4 6 3 1 3 9 1 7 6 8 4 3 1 4 0 5 3 6 9 6 1 7 5 4
 4 7 2 8 2 2 5 7 9 5 4 8 8 4 9 0 8 9 8 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7
 8 9 0 1 2 3 4 5 6 7 8 9 0 9 5 5 6 5 0 9 8 9 8 4 1 7 7 3 5 1 0 0]

Features of first five samples of digits data:
[[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]
 [ 0.  0.  

In [703]:
#How many labels does digits data have? 
#print the unique classes
print('unique classes: ',np.unique(digits.target,return_counts=True))
n = digits.data.shape[0] #Number of data points or samples 
d = digits.data.shape[1] #Dimension of data points
print(n,d)

unique classes:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([178, 182, 177, 183, 181, 182, 181, 179, 174, 180], dtype=int64))
1797 64


- Here we can see that all the classes have been balanced

In [704]:
'''
# Shuffle dataset
indexarr = np.arange(n) #index array
np.random.shuffle(indexarr) #shuffle the indices 
'''

'\n# Shuffle dataset\nindexarr = np.arange(n) #index array\nnp.random.shuffle(indexarr) #shuffle the indices \n'

- Choosing 80 % of the samples for train set using KNN Classifier from sklearn

In [705]:
#Use the samples corresponding to first 80% of indexarr for training 
num_train = int(0.8*n)
#Use the remaining 20% samples for testing 
num_test = n-num_train
print('num_train: ',num_train, 'num_test: ', num_test)

num_train:  1437 num_test:  360


In [706]:
#Use the first 80% of indexarr to create the train data features and train labels 
train_features = A[0:num_train]#indexarr
train_label = y[0:num_train]#indexarr
print('shape of train data features:')
print(train_features.shape)
print('shape of train data labels')
print(train_label[0:140])

shape of train data features:
(1437, 64)
shape of train data labels
[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 9 5 5 6 5 0
 9 8 9 8 4 1 7 7 3 5 1 0 0 2 2 7 8 2 0 1 2 6 3 3 7 3 3 4 6 6 6 4 9 1 5 0 9
 5 2 8 2 0 0 1 7 6 3 2 1 7 4 6 3 1 3 9 1 7 6 8 4 3 1 4 0 5 3 6 9 6 1 7 5 4
 4 7 2 8 2 2 5 7 9 5 4 8 8 4 9 0 8 9 8 0 1 2 3 4 5 6 7 8 9]


In [707]:
#Use remaining 20% of indexarr to create the test data and test labels  
test_features = A[num_train:n]
test_label = y[num_train:n]
print('shape of test data features:')
print(test_features.shape)
print('shape of test data labels')
print(test_label.shape)

shape of test data features:
(360, 64)
shape of test data labels
(360,)


### Noisy samples being generated for 10% of the training data.

In [708]:
# Here I have taken 200 samples to be made noisy
for i in range(200):
    temp=train_features[i,:]
    print(temp)
    for i in range(len(temp)):
        temp[i]+=random.uniform(-1,1) # Changes to dataset using epsilon for the range of +_ (0,1)
    #print(temp)
    train_features[i,:]=temp
    print(train_features[i,:])

[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
[ 3.76219619  3.22997238  7.5494889  16.66665235 12.40985947  4.78149702
  2.53414428  3.45240257  3.49119628  3.89311647 16.76079766 18.89487081
 12.40186424 17.38631683  7.31487072  3.98193146  2.26450137  5.50460296
 18.77014215  5.97929383  2.40890003 14.96232063 10.25603805  2.5820049
  2.84989173  6.68734276 15.00323407  3.36668257  2.13929223 10.15658045
 10.52346889  3.32644802  3.79658257  7.9968913  10.29400649  3.15199863
  3.82235419 11.40312149 11.20472216  3.51665096  2.6344576   7.77660564
 14.19903678  2.86785035  3.30176334 15.15965974  9.12335441  3.95681176
  2.99266626  5.30140918 16.84322085  7.86054413 12.42432618 14.81868513
  3.3079761   2.76038422  3.68651696  3.94311165  9.5038791  16.60812108
 13.454081

- Prediction of KNN with noisy data. Here the value of k has been kept large otherwise for k=1 the accuracy on test set is 0.999

In [709]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
# Define parameter grid to search over
param_grid = {'n_neighbors': [31,45]} # Such a high number is to show presence of noise otherwise 0.99 % accuracy is coming up

# Perform 5-fold cross-validation with stratified sampling to balance all the classes in the cross validation set
cv = StratifiedKFold(n_splits=5)

# SVC kernel 
lr = KNeighborsClassifier() 

# Perform grid search over parameter grid using cross-validation
grid_search_knn = GridSearchCV(lr, param_grid, cv=cv, scoring='accuracy')

# Fit the grid search to the training data
grid_search_knn.fit(train_features,train_label)

y_pred_train=grid_search_knn.predict(train_features)
y_pred_test=grid_search_knn.predict(test_features)

# Print the best hyperparameters and accuracy score,precision,recall,specificity and sensitivity
print("Best hyperparameters: ", grid_search_knn.best_params_)

print("\nAccuracy")
print("Training Set: ", grid_search_knn.score(train_features,train_label))
print("Test Set: ", grid_search_knn.score(test_features,test_label))

Best hyperparameters:  {'n_neighbors': 31}

Accuracy
Training Set:  0.9749478079331941
Test Set:  0.9638888888888889


### Now to remove noise

In [710]:
k=[1,2,3,4,5,6,7,8,9,10,11] # taking a list of k
removed=0
new_train_features=[]
new_train_labels=[]
for i in range(num_train):
    c=0
    for j in k:
        lr = KNeighborsClassifier(n_neighbors=j) 
        lr.fit(train_features,train_label)
        #print(train_features.shape)
        if(lr.predict(train_features[i,:].reshape(1,64))==train_label[i]):
            c+=1
    if((c/len(k))<1):
        removed+=1
        #print('Noisy feature index:',i)
        continue
    else:
        new_train_features.append(train_features[i,:])
        new_train_labels.append(train_label[i])


### COMMENTS :

- In this case I have used multiple k values ranging from 1 to 11. This has been done for ensembling method.

- Small values of k has been chosen as the k value if it is smaller is more prone to noisy data whereas the larger values of k smotthen the effect of noise.

- Here the algorithm which I have used is that I am evaluating the predicted label from the KNN clusters given over a large values of k. If the true label becomes not equal to the predicted label and the total sum is less than a particular threshold then I am dropping that Data point.

In [711]:
print('Number of samples removed',removed)

Number of samples removed 27


In [712]:
lr = KNeighborsClassifier(n_neighbors=31) 
lr.fit(new_train_features,new_train_labels)
print(lr.score(new_train_features,new_train_labels))
print(lr.score(test_features,test_label))

0.9886524822695035
0.9694444444444444


### COMMENTS :

- This methods appears to work in the above case to improve the accuracy of the test set as well the train set. However a more rigorous testing is required to ensure that this algorithm works in all cases.

- A proper choice of k has to be made and a much more rigorous approach is required in selecting all values of k.